# Contour plot with colorbar


In [ ]:
import numpy as np

import matplotlib as mpl
from mpllayout import geometry as geo
from mpllayout.solver import solve
from mpllayout.layout import Layout

from mpllayout.matplotlibutils import subplots

In [ ]:
# Use this so matplotlib figures are showed with whitespace
%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

## Create a layout

In [ ]:
## Create the figure and two axes (one for the contour plot and one for the colorbar)

### Primitives
# Create a `Layout` to track all geometric primitives and constraints
layout = Layout()

# Create the main geometric primitives
layout.add_prim(pr.Quadrilateral(), "Figure")
layout.add_prim(pr.Axes(), "AxesContour")
layout.add_prim(pr.Axes(), "AxesColorbar")

### Constraints

# Make all the quadrilaterals rectangular
layout.add_constraint(geo.Box(), ("Figure",), ())
layout.add_constraint(geo.Box(), ("AxesContour/Frame",), ())
layout.add_constraint(geo.Box(), ("AxesColorbar/Frame",), ())

## Figure size
# Set the figure width
layout.add_constraint(geo.Length(), ("Figure/Line0",), (5,))

# Fix the figure's lower left corner to the origin
layout.add_constraint(geo.Fix(), ("Figure/Line0/Point0",), (np.array([0, 0]),))

## Axes sizes

# Set the aspect ratio to match physical dimensions
layout.add_constraint(
    geo.RelativeLength(), ("AxesContour/Frame/Line0", "AxesContour/Frame/Line1"), (2,)
)

# Set the color bar height to 1/4 inch
layout.add_constraint(geo.Length(), ("AxesColorbar/Frame/Line1",), (1 / 8,))

## Align the color bar with the contour plot
# Make the left/right sides collinear
layout.add_constraint(
    geo.Collinear(), ("AxesContour/Frame/Line3", "AxesColorbar/Frame/Line3"), ()
)
layout.add_constraint(
    geo.Collinear(), ("AxesContour/Frame/Line1", "AxesColorbar/Frame/Line1"), ()
)

## Margins
# Place the color bar 1/16 inch above the contour plot
layout.add_constraint(
    geo.MidpointYDistance(),
    ("AxesContour/Frame/Line2", "AxesColorbar/Frame/Line0"),
    (1 / 16,)
)

# Set the left margin to 6/8 inch from the contour plot
layout.add_constraint(
    geo.MidpointXDistance(), ("Figure/Line3", "AxesContour/Frame/Line3"), (6 / 8,)
)

# Set the right margin 1/8 inch from the contour plot
layout.add_constraint(
    geo.MidpointXDistance(), ("AxesContour/Frame/Line1", "Figure/Line1"), (1 / 8,)
)

# Set the top margin to 1/2 inch
layout.add_constraint(
    geo.MidpointYDistance(), ("AxesColorbar/Frame/Line2", "Figure/Line2"), (1 / 2,)
)

# Set the bottom margin to 6/8 inch
layout.add_constraint(
    geo.MidpointYDistance(), ("Figure/Line0", "AxesContour/Frame/Line0"), (6 / 8,)
)

In [ ]:
## Solve the constrained system of plots

root_prim_n, solve_info = solve(layout)

fig, axs = subplots(root_prim_n)
# The sizes of `fig` and axes in `axs` will reflect the constraints
# `axs['AxesColorbar']` is the colorbar and `axs['AxesContour']` is the contour


print(fig.get_size_inches())

x = np.linspace(0, 10, 51)
y = np.linspace(0, 5, 26)
xx, yy = np.meshgrid(x, y)
z = (xx - 5) ** 2 + (yy - 2.5) ** 2

cset = axs["AxesContour"].contourf(x, y, z)

axs["AxesContour"].set_xlabel("x [cm]")
axs["AxesContour"].set_ylabel("y [cm]")

fig.colorbar(cset, cax=axs["AxesColorbar"], orientation="horizontal")
axs["AxesColorbar"].xaxis.set_label_text("z [cm]")
axs["AxesColorbar"].xaxis.set_tick_params(
    top=True, labeltop=True, bottom=False, labelbottom=False
)
axs["AxesColorbar"].xaxis.set_label_position(position="top")

fig.savefig("contour_colorbar.png")